In [1]:
%load_ext autoreload
%autoreload 2

# Training as a function of the number of patterns

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from wassa.wassa_plots import plot_results_std, plot_SM, plot_colored_raster
from wassa.dataset_generation import sm_generative_model, generate_dataset
from wassa.wassa_utils import performance_as_a_function_of_number_of_motifs

In [3]:
date = '2024_01_15'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
class dataset_parameters():
    seed = 666
    
    N_pre = 100 # number of neurons
    N_timesteps = 255 # number of timesteps for the raster plot (in ms)
    N_samples = 100 # total number of samples in the dataset

    N_delays = 51 # number of timesteps in spiking motifs, must be a odd number for convolutions
    N_SMs = 1 # number of structured spiking motifs
    N_involved = N_pre*torch.ones(N_SMs) # number of neurons involved in the spiking motif
    avg_fr = 20 # average firing rate of the neurons (in Hz)
    std_fr = .1 # standard deviation for the firing rates of the different neurons
    frs = torch.normal(avg_fr, std_fr, size=(N_pre,)).abs()
    freq_sms = 16*torch.ones(N_SMs) # frequency of apparition of the different spiking motifs (in Hz)
    overlapping_sms = False # possibility to have overlapping sequences

    temporal_jitter = .1 # temporal jitter for the spike generation in motifs
    dropout_proba = 0 # probabilistic participations of the different neurons to the spiking motif
    additive_noise = .1 # percentage of background noise/spontaneous activity
    warping_coef = 1 # coefficient for time warping

    def get_parameters(self):
        return f'{self.N_pre}_{self.N_delays}_{self.N_SMs}_{self.N_timesteps}_{self.N_samples}_{self.N_involved.mean()}_{self.avg_fr}_{self.freq_sms.mean()}_{self.overlapping_sms}_{self.temporal_jitter}_{self.dropout_proba}_{self.additive_noise}_{self.warping_coef}_{self.seed}'

torch.serialization.add_safe_globals([dataset_parameters])

In [5]:
class training_parameters:
    kernel_size = (dataset_parameters.N_SMs, dataset_parameters.N_pre, dataset_parameters.N_delays)
    loss_type = 'mse'
    N_learnsteps = 1000
    learning_rate = .001
    penalty_type = 'smoothed_orthogonality'
    smoothwind = 40
    lambda_ = .014
    batch_size = None
    output = 'linear' 
    do_bias = True 
    zeros = 'ignore'
    wass_order = 1
    weight_init = None
    if not penalty_type: 
        lambda_ = 0
    elif penalty_type[:8] != 'smoothed': 
        smoothwind = 0
    if lambda_ == 0:
        penalty_type = None
    def get_parameters(self):
        name = f'{self.loss_type}_{self.output}_{self.penalty_type}_{self.do_bias}_{self.kernel_size}_{self.N_learnsteps}_{self.learning_rate}_{self.lambda_}_{self.batch_size}_{self.smoothwind}'
        if self.loss_type == 'emd':
            name += f'_{self.zeros}_{self.wass_order}'
        return name

In [6]:
params_mse = training_parameters()
params_emd = training_parameters()
params_emd.loss_type = 'emd'
params_emd.penalty_type = 'kernels_orthogonality'
params_emd.lambda_ = .0014

In [ ]:
N_iter = 10
seeds = torch.arange(0,N_iter)
num_patterns = torch.arange(1,10)
results, num_patterns = performance_as_a_function_of_number_of_motifs(dataset_parameters, params_emd, params_mse, date, num_patterns, N_iter = N_iter, seeds = seeds, device=device)

results/2024_01_15_performance_as_a_function_of_number_of_motifs_100_51_1_255_100_100.0_20_16.0_False_0.1_0_0.1_1_666_emd_linear_kernels_orthogonality_True_(1, 100, 51)_1000_0.001_0.0014_None_40_ignore_1_1_9


/home/antoineg/miniconda3/envs/wassa/lib/python3.12/site-packages/wassa-1.0-py3.12.egg/wassa/wassa_utils.py:22: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /opt/conda/conda-bld/pytorch_1729647175880/work/aten/src/ATen/native/Convolution.cpp:1036.)
  2%|███▎                                                                                                                                               | 2/90 [1:42:04<69:02:20, 2824.32s/it]

In [ ]:
def plot_results_std(ax, results, coefs, xlabel, ylabel, legend, color, do_ylabel, ymax=None, ymin=None, do_legend=False, logplot=False, quantile=False):
     
    mean_, std_ = results.mean(axis=0), results.std(axis=0)
    
    if quantile: 
        q5 = results.quantile(.1,axis=0)
        q95 = results.quantile(.9,axis=0)

    if quantile:
        bottom_ = q5
    elif ymin is not None:
        bottom_ = np.maximum(mean_ - std_, ymin*np.ones([len(mean_)]))
    else:
        bottom_ = mean_ - std_
        
    if quantile:
        top_ = q95
    elif ymax is not None:
        top_= np.minimum(mean_ + std_, ymax*np.ones([len(mean_)]))
    else:
        top_ = mean_ + std_

    if logplot:
        ax.semilogx(coefs, mean_, '.',color=color, label=legend)
    else:
        ax.plot(coefs, mean_, '.',color=color, label=legend)
    ax.fill_between(coefs, bottom_, top_, facecolor=color, edgecolor=None, alpha=.3)

    ax.set_ylim(ymin,ymax)
    
    if xlabel: 
        ax.set_xlabel(xlabel, fontsize=14)
    else: 
        ax.set_xticks([])
    if do_ylabel:
        ax.set_ylabel('similarity valuseu jorge albume', fontsize=14)
    else: 
        ax.set_yticks([])
    ax.set_title(ylabel, fontsize=16)
    if do_legend: 
        ax.legend(fontsize=12);

    return ax

In [ ]:
import matplotlib.pyplot as plt

name_metrics = ['factors similarity', 'kernels similarity', 'mean timings similarity']
name_methods = ['MSE', 'EMD', 'combined']
colors = ['darkolivegreen','blue','orangered']
xlabel = '# spiking motifs'

results = results.cpu()
fig, ax = plt.subplots(1,3, figsize=(16,5))
for i in range(len(name_metrics)):
    for m in range(len(name_methods)):
        if i==0:
            do_ylabel = True
        else:
            do_ylabel = False
        ax[i] = plot_results_std(ax[i],results[m,:,:,i],num_patterns,xlabel,name_metrics[i],name_methods[m],colors[m],ymax=1,ymin=0,do_ylabel=do_ylabel,quantile=True)